In [1]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import spacy
from transformers import BertTokenizer, BertModel

from pykospacing import Spacing


# tqdm과 pandas 통합
tqdm.pandas()

# 한국어 모델 로드
nlp = spacy.load("ko_core_news_sm")

c:\Anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 불용어 파일 로드
stopwords_file = '../../../../../data/stopwords-ko.txt'
with open(stopwords_file, 'r', encoding='utf-8') as f:
    stopwords = set(line.strip() for line in f if line.strip())

# 불용어 처리 함수 정의
def remove_stopwords(text, stopwords):
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if token.text not in stopwords and not token.is_punct and not token.is_space]
    return filtered_tokens


spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def text_to_vector(text):
    inputs = tokenizer(text, return_tensors='pt')           # 입력 텍스트를 토큰화
    with torch.no_grad():                                   # 모델에 입력하여 출력 벡터 얻기
        outputs = model(**inputs)
    cls_vector = outputs.last_hidden_state[0][0].numpy()    # [CLS] 토큰에 대한 벡터 추출
    return cls_vector

In [24]:
df = pd.read_excel('../../../../../data/filtered_news.xlsx')

# 데이터 전처리 및 토큰화
# df = df[:100]
df['cleaned'] = df['feature'].apply(lambda x: remove_stopwords(preprocessing(x), stopwords))

In [25]:
# KoBERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 텍스트를 벡터로 변환하여 데이터프레임에 추가
df['vector'] = df['cleaned'].apply(text_to_vector)